In [23]:
import numpy as np 
import pandas as pd 
from io import open
import tensorflow as tf
import glob
import pickle
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [3]:
!pip install music21

     |████████████████████████████████| 19.2 MB 807 kB/s eta 0:00:01
  Created wheel for music21: filename=music21-6.7.1-py3-none-any.whl size=21941692 sha256=77ccb12f6933bf75c9340057fab7b46745fc9b2d3dd860368c831144acbaf7cb
  Stored in directory: /root/.cache/pip/wheels/72/44/61/90e4e65262ca1b4d9f707527b540729ce3f64e00fc6b38d54c
Successfully built music21


In [4]:
from music21 import converter, instrument, note, chord, stream

In [11]:
def preprocess_input(filename, folder=False):
    # master list of notes
    notes = []
    
    # converting folders with multiple MIDI files
    if folder == True:
        assert os.path.exists('../input/classical-music-midi/'+filename)
        for file in glob.glob('../input/classical-music-midi/'+filename+'/*.mid'):
            notes_per_piece = []
            # read the MIDI file
            midi = converter.parse(file)            
            
            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[1].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes
            
#             print(notes_to_parse)
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes_per_piece.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes_per_piece.append('.'.join(str(n) for n in element.normalOrder))
            notes.append(notes_per_piece)
    else:
        assert os.path.exists(filename)
        midi = converter.parse(filename)
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[1].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
#         print(notes_to_parse)
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        

    with open('./notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
#     print(notes)
    return notes

In [10]:
notes = preprocess_input('bach', folder=True)

[['C5',
  'C3',
  'E-4',
  'G3',
  'D4',
  'F3',
  'E-4',
  'G3',
  'C4',
  'E-3',
  'E-4',
  'G3',
  'D4',
  'F3',
  'E-4',
  'G3',
  'C5',
  'C3',
  'E-4',
  'G3',
  'D4',
  'F3',
  'E-4',
  'G3',
  'C4',
  'E-3',
  'E-4',
  'G3',
  'D4',
  'F3',
  'E-4',
  'G3',
  'G#4',
  'C3',
  'F4',
  'G#3',
  'E4',
  'G3',
  'F4',
  'G#3',
  'C4',
  'F3',
  'F4',
  'G#3',
  'E4',
  'G3',
  'F4',
  'G#3',
  'G#4',
  'C3',
  'F4',
  'G#3',
  'E4',
  'G3',
  'F4',
  'G#3',
  'C4',
  'F3',
  'F4',
  'G#3',
  'E4',
  'G3',
  'F4',
  'G#3',
  'B4',
  'C3',
  'F4',
  'G#3',
  'E-4',
  'G3',
  'F4',
  'G#3',
  'D4',
  'F3',
  'F4',
  'G#3',
  'E-4',
  'G3',
  'F4',
  'G#3',
  'B4',
  'C3',
  'F4',
  'G#3',
  'E-4',
  'G3',
  'F4',
  'G#3',
  'D4',
  'F3',
  'F4',
  'G#3',
  'E-4',
  'G3',
  'F4',
  'G#3',
  'C5',
  'C3',
  'G4',
  'E-3',
  'F4',
  'D3',
  'G4',
  'E-3',
  'E-4',
  'G3',
  'G4',
  'E-3',
  'F4',
  'D3',
  'G4',
  'E-3',
  'C5',
  'C3',
  'G4',
  'E-3',
  'F4',
  'D3',
  'G4',
  'E-3',
 

In [12]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(preprocess_input('../input/classical-music-midi/bach/bach_846.mid'))
        self.valid = self.tokenize(preprocess_input('../input/classical-music-midi/bach/bach_847.mid'))
        self.test = self.tokenize(preprocess_input('../input/classical-music-midi/bach/bach_850.mid'))

    def tokenize(self, notes):
        """Tokenizes a note sequence"""
        assert len(notes) > 0
        
        # Add notes to the dictionary
        for note in notes:
            self.dictionary.add_word(note)
#         # Add words to the dictionary
#         with open(path, 'r', encoding="utf8") as f:
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 for word in words:
#                     self.dictionary.add_word(word)

        # Tokenize file content
        idss = []
        ids = []
        for note in notes:
            ids.append(self.dictionary.word2idx[note])
        idss.append(torch.tensor(ids).type(torch.int64))
        ids = torch.cat(idss)
            
#         with open(path, 'r', encoding="utf8") as f:
#             idss = []
#             for line in f:
#                 words = line.split() + ['<eos>']
#                 ids = []
#                 for word in words:
#                     ids.append(self.dictionary.word2idx[word])
#                 idss.append(torch.tensor(ids).type(torch.int64))
#             ids = torch.cat(idss)

        return ids

In [17]:
corpus = Corpus('../input/classical-music-midi/bach')

In [22]:
device = 'cuda'
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, eval_batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

In [ ]:
# This recognises the events in a MIDI track
# The type of events inclde: playing a note, stopping a note, or another system executive instruction
class MIDIEvent:

    # defining an enumeration on the type of MIDI events
    class Type(Enum):
        noteOFF = 0
        noteON = 1
        other = 2

    # A MIDI event has the following features
    # Type = The type of event (from the above enumeration)
    # Key = The note being played
    # velocity = the speed of the note in the track
    # deltaTick = the time difference between this and the previous event
    def __init__(self, note, noteID=0, vel=0, delta=0):
        self.type = note
        self.key = noteID
        self.velocity = vel
        self.deltaTick = delta

    def __repr__(self):
        return ("\nEvent Type: " + str(self.type) + " Key: " + str(self.key) +
                " Velocity: " + str(self.velocity) + " delta tick: " +
                str(self.deltaTick))

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 1000):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src